In [1]:
import os
import tensorflow  as tf
import time

In [2]:
def decode(serialized_example):
    alphabets=list()
    features = tf.parse_single_example(
      serialized_example,      
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'cho_label':tf.FixedLenFeature([], tf.int64),
          'jung_label':tf.FixedLenFeature([], tf.int64),
          'jong_label':tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'height': tf.FixedLenFeature([], tf.int64),
          'image': tf.FixedLenFeature([], tf.string)
      })
    image = tf.decode_raw(features['image'],tf.uint8)
    #image = tf.reshape(image,[64,64,1])
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [64,64,1])
    label = tf.cast(features['label'],tf.int64)
    cho_label=tf.cast(features['cho_label'],tf.int64)
    jung_label=tf.cast(features['jung_label'],tf.int64)
    jong_label=tf.cast(features['jong_label'],tf.int64)
    
    return image,label, cho_label, jung_label, jong_label

In [3]:
batch_size=100
test_file_name = "Shuffled_with_alphabet_PE92_Test.tfrecord"
train_file_name = "Shuffled_with_alphabet_PE92_Train.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

In [4]:
train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label, train_cho, train_jung, train_jong = train_iterator.get_next()
test_image, test_label, test_cho, test_jung, test_jong = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [5]:
input_images = tf.placeholder(tf.float32,[None,64,64,1])
input_label_cho = tf.placeholder(tf.int64,[None])
input_label_jung = tf.placeholder(tf.int64,[None])
input_label_jong = tf.placeholder(tf.int64,[None])
input_labels= tf.placeholder(tf.int64,[None])


is_training = tf.placeholder(tf.bool)
keep_prob = tf.placeholder(tf.float32)
print(input_images)


Tensor("Placeholder:0", shape=(?, 64, 64, 1), dtype=float32)


In [6]:
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L1 = tf.nn.conv2d(input_images,W1, strides=[1,1,1,1], padding = 'SAME')
print(L1)
bn1 = tf.layers.batch_normalization(L1, training=is_training)
at1 = tf.nn.relu(bn1)

pool1 =  tf.nn.max_pool(at1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool1)

Tensor("Conv2D:0", shape=(?, 64, 64, 32), dtype=float32)
Tensor("MaxPool:0", shape=(?, 32, 32, 32), dtype=float32)


In [7]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))

L2 = tf.nn.conv2d(pool1,W2, strides=[1,1,1,1], padding = 'SAME')
bn2 = tf.layers.batch_normalization(L2, training=is_training)
at2 = tf.nn.relu(bn2)

pool2 =  tf.nn.max_pool(at2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool2)

Tensor("MaxPool_1:0", shape=(?, 16, 16, 64), dtype=float32)


In [8]:
W3 = tf.Variable(tf.random_normal([4, 4, 64, 128], stddev=0.01))

L3 = tf.nn.conv2d(pool2,W3, strides=[1,1,1,1], padding = 'SAME')
bn3 = tf.layers.batch_normalization(L3, training=is_training)
at3 = tf.nn.relu(bn3)

pool3 =  tf.nn.max_pool(at3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool3)

Tensor("MaxPool_2:0", shape=(?, 8, 8, 128), dtype=float32)


In [9]:
W4 = tf.Variable(tf.random_normal([4, 4, 128, 256], stddev=0.01))

L4 = tf.nn.conv2d(pool3,W4, strides=[1,1,1,1], padding = 'SAME')
bn4 = tf.layers.batch_normalization(L4, training=is_training)
at4 = tf.nn.relu(bn4)

pool4 =  tf.nn.max_pool(at4, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

W5 = tf.Variable(tf.random_normal([2, 2, 256, 512], stddev=0.01))
L5 = tf.nn.conv2d(pool4,W5, strides=[1,1,1,1], padding = 'SAME')
bn5 = tf.layers.batch_normalization(L5, training=is_training)
at5 = tf.nn.relu(bn5)
pool5 =  tf.nn.max_pool(at5, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')

pool6 =  tf.nn.max_pool(pool5, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(pool4)
print(pool6)

Tensor("MaxPool_3:0", shape=(?, 4, 4, 256), dtype=float32)
Tensor("MaxPool_5:0", shape=(?, 1, 1, 512), dtype=float32)


In [10]:
fc = tf.reshape(pool6,[-1,512])

W_cho1 = tf.Variable(tf.random_normal([512, 48], stddev=0.01))
W_jung1 = tf.Variable(tf.random_normal([512, 48], stddev=0.01))
W_jong1 = tf.Variable(tf.random_normal([512, 48], stddev=0.01))

fc_cho = tf.matmul(fc, W_cho1)
fc_cho = tf.layers.batch_normalization(fc_cho, training=is_training)
fc_cho = tf.nn.relu(fc_cho)
fc_cho= tf.nn.dropout(fc_cho,keep_prob)

fc_jung = tf.matmul(fc, W_jung1)
fc_jung = tf.layers.batch_normalization(fc_jung, training=is_training)
fc_jung = tf.nn.relu(fc_jung)
fc_jung= tf.nn.dropout(fc_jung,keep_prob)

fc_jong = tf.matmul(fc, W_jong1)
fc_jong = tf.layers.batch_normalization(fc_jong, training=is_training)
fc_jong = tf.nn.relu(fc_jong)
fc_jong= tf.nn.dropout(fc_jong,keep_prob)
print(fc)

Tensor("Reshape:0", shape=(?, 512), dtype=float32)


In [11]:
W_cho2 = tf.Variable(tf.random_normal([48,19],stddev=0.01))
W_jung2 = tf.Variable(tf.random_normal([48,21],stddev=0.01))
W_jong2 = tf.Variable(tf.random_normal([48,28],stddev=0.01))

logit_cho = tf.matmul(fc_cho, W_cho2)
logit_jung = tf.matmul(fc_jung, W_jung2)
logit_jong = tf.matmul(fc_jong, W_jong2)

print(logit_cho)
print(logit_jung)
print(logit_jong)

Tensor("MatMul_3:0", shape=(?, 19), dtype=float32)
Tensor("MatMul_4:0", shape=(?, 21), dtype=float32)
Tensor("MatMul_5:0", shape=(?, 28), dtype=float32)


In [12]:
cross_entropy_cho = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_cho, labels=input_label_cho)
cross_entropy_jung = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_jung, labels=input_label_jung)
cross_entropy_jong = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit_jong, labels=input_label_jong)

loss_cho = tf.reduce_mean(cross_entropy_cho)
cho_loss = tf.summary.scalar("Loss_cho", loss_cho)
loss_jung = tf.reduce_mean(cross_entropy_jung)
jung_loss = tf.summary.scalar("Loss_jung",loss_jung)
loss_jong = tf.reduce_mean(cross_entropy_jong)
jong_loss = tf.summary.scalar("Loss_jong",loss_jong)

total_loss = 3*((loss_cho*0.4) + (loss_jung*0.4) + (loss_jong*0.2))

update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    optimizer = tf.train.AdamOptimizer(0.0005).minimize(total_loss)
loss_summ=tf.summary.scalar("Loss",total_loss)
print(total_loss)

Tensor("mul_3:0", shape=(), dtype=float32)


In [13]:
argmax_cho = tf.argmax(logit_cho,1)
argmax_jung = tf.argmax(logit_jung,1)
argmax_jong = tf.argmax(logit_jong,1)
print(argmax_cho)
prediction = tf.stack([argmax_cho,argmax_jung,argmax_jong],1)

Tensor("ArgMax:0", shape=(?,), dtype=int64)


In [14]:
total_labels = tf.stack([input_label_cho, input_label_jung, input_label_jong],1)
equal_check = tf.equal(prediction,total_labels)
cast_equal = tf.cast(equal_check, tf.float32)
reduce_min = tf.reduce_min(cast_equal,1)

accuracy = tf.reduce_mean(reduce_min)
test_accuracy = tf.reduce_sum(reduce_min)
acc_summ=tf.summary.scalar("Accuracy",accuracy)

cho_equal = tf.equal(argmax_cho, input_label_cho)
jung_equal = tf.equal(argmax_jung, input_label_jung)
jong_equal = tf.equal(argmax_jong, input_label_jong)

cho_acc = tf.reduce_mean(tf.cast(cho_equal,tf.float32))
cho_summ = tf.summary.scalar("Cho_Acc", cho_acc)
jung_acc = tf.reduce_mean(tf.cast(jung_equal,tf.float32))
jung_summ = tf.summary.scalar("Jung_Acc", cho_acc)
jong_acc = tf.reduce_mean(tf.cast(jong_equal,tf.float32))
jong_summ = tf.summary.scalar("Jong_Acc", cho_acc)

batch_test_cho_acc = tf.reduce_sum(tf.cast(cho_equal,tf.float32))
batch_test_jung_acc = tf.reduce_sum(tf.cast(jung_equal,tf.float32))
batch_test_jong_acc = tf.reduce_sum(tf.cast(jong_equal,tf.float32))

In [15]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    
    merged_summary = tf.summary.merge([loss_summ, acc_summ,cho_summ,jung_summ,jong_summ])
    train_writer = tf.summary.FileWriter("./logs/CNNV2/PE92/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV2/PE92/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    
    #Train Model
    
    start_time = time.time()
    global_step=0
    
    for epoch in range(50):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_cho, batch_jung, batch_jong = sess.run([train_image, train_cho, train_jung, train_jong])
            
            sess.run(optimizer,feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:True})
            train_acc, train_loss, train_cho_acc, train_jung_acc, train_jong_acc = sess.run([accuracy, total_loss,cho_acc, jung_acc, jong_acc ], feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                #print("  **Acc cho: %f, Acc jung: %f, Acc jong: %f **"% (train_cho_acc,train_jung_acc, train_jong_acc))
                summary = sess.run(merged_summary, feed_dict={input_images: batch_images, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
                
        
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        save_path = saver.save(sess, "save/PE92/CNNV2/CNNV2_PE92.ckpt")
        
        acc_sum=0.0
        loss_sum=0.0
        cho_sum=0.0
        jung_sum=0.0
        jong_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_cho, test_jung, test_jong])
            try:
                test_acc, test_loss, test_cho_acc, test_jung_acc, test_jong_acc = sess.run([test_accuracy, total_loss, batch_test_cho_acc, batch_test_jung_acc, batch_test_jong_acc ], feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})

            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
            cho_sum+=test_cho_acc
            jung_sum+=test_jung_acc
            jong_sum+=test_jong_acc
            loss_sum+=test_loss*len(batch_cho)
            
        test_loss=loss_sum/test_data_length
        test_acc = acc_sum/test_data_length
        test_cho_acc = cho_sum/test_data_length
        test_jung_acc = jung_sum/test_data_length
        test_jong_acc = jong_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
       # print("  Acc cho: %f, Acc jung: %f, Acc jong: %f "% (test_cho_acc,test_jung_acc, test_jong_acc))

    print("Training is done.")


    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(threads)

Train total Batch :2002
Epoch :0 | step 0, training accuracy: 0.000000, train loss: 9.185826
Epoch :0 | step 100, training accuracy: 0.050000, train loss: 8.499432
Epoch :0 | step 200, training accuracy: 0.020000, train loss: 6.252926
Epoch :0 | step 300, training accuracy: 0.050000, train loss: 4.687982
Epoch :0 | step 400, training accuracy: 0.220000, train loss: 3.751001
Epoch :0 | step 500, training accuracy: 0.400000, train loss: 3.080655
Epoch :0 | step 600, training accuracy: 0.430000, train loss: 3.040377
Epoch :0 | step 700, training accuracy: 0.640000, train loss: 1.666927
Epoch :0 | step 800, training accuracy: 0.680000, train loss: 1.449079
Epoch :0 | step 900, training accuracy: 0.570000, train loss: 1.566273
Epoch :0 | step 1000, training accuracy: 0.700000, train loss: 1.060702
Epoch :0 | step 1100, training accuracy: 0.690000, train loss: 1.230569
Epoch :0 | step 1200, training accuracy: 0.670000, train loss: 1.109408
Epoch :0 | step 1300, training accuracy: 0.830000, t

Epoch :5 | step 500, training accuracy: 0.980000, train loss: 0.063772
Epoch :5 | step 600, training accuracy: 0.990000, train loss: 0.068499
Epoch :5 | step 700, training accuracy: 0.980000, train loss: 0.109442
Epoch :5 | step 800, training accuracy: 0.940000, train loss: 0.225070
Epoch :5 | step 900, training accuracy: 0.990000, train loss: 0.074311
Epoch :5 | step 1000, training accuracy: 0.920000, train loss: 0.183935
Epoch :5 | step 1100, training accuracy: 0.980000, train loss: 0.079058
Epoch :5 | step 1200, training accuracy: 0.980000, train loss: 0.064576
Epoch :5 | step 1300, training accuracy: 0.960000, train loss: 0.131745
Epoch :5 | step 1400, training accuracy: 0.980000, train loss: 0.081118
Epoch :5 | step 1500, training accuracy: 0.930000, train loss: 0.239623
Epoch :5 | step 1600, training accuracy: 0.940000, train loss: 0.226805
Epoch :5 | step 1700, training accuracy: 0.980000, train loss: 0.197340
Epoch :5 | step 1800, training accuracy: 0.980000, train loss: 0.0469

Epoch :10 | step 1000, training accuracy: 1.000000, train loss: 0.009045
Epoch :10 | step 1100, training accuracy: 1.000000, train loss: 0.007710
Epoch :10 | step 1200, training accuracy: 0.990000, train loss: 0.056447
Epoch :10 | step 1300, training accuracy: 0.970000, train loss: 0.099395
Epoch :10 | step 1400, training accuracy: 0.990000, train loss: 0.083599
Epoch :10 | step 1500, training accuracy: 0.990000, train loss: 0.016805
Epoch :10 | step 1600, training accuracy: 0.950000, train loss: 0.129748
Epoch :10 | step 1700, training accuracy: 0.980000, train loss: 0.034508
Epoch :10 | step 1800, training accuracy: 1.000000, train loss: 0.024278
Epoch :10 | step 1900, training accuracy: 0.960000, train loss: 0.177447
Epoch :10 | step 2000, training accuracy: 1.000000, train loss: 0.031864
Time : 0 : 35: :10.301702
Test Acc : 0.959831, Test Loss :0.204513
Epoch :11 | step 0, training accuracy: 0.990000, train loss: 0.046849
Epoch :11 | step 100, training accuracy: 0.990000, train los

Epoch :15 | step 1400, training accuracy: 0.990000, train loss: 0.042905
Epoch :15 | step 1500, training accuracy: 0.980000, train loss: 0.043409
Epoch :15 | step 1600, training accuracy: 1.000000, train loss: 0.015854
Epoch :15 | step 1700, training accuracy: 0.990000, train loss: 0.028298
Epoch :15 | step 1800, training accuracy: 0.990000, train loss: 0.055142
Epoch :15 | step 1900, training accuracy: 1.000000, train loss: 0.015128
Epoch :15 | step 2000, training accuracy: 0.980000, train loss: 0.038033
Time : 0 : 51: :5.208320
Test Acc : 0.958848, Test Loss :0.234846
Epoch :16 | step 0, training accuracy: 1.000000, train loss: 0.008917
Epoch :16 | step 100, training accuracy: 1.000000, train loss: 0.007304
Epoch :16 | step 200, training accuracy: 1.000000, train loss: 0.022784
Epoch :16 | step 300, training accuracy: 0.990000, train loss: 0.020373
Epoch :16 | step 400, training accuracy: 0.990000, train loss: 0.017730
Epoch :16 | step 500, training accuracy: 0.970000, train loss: 0.

Epoch :20 | step 1800, training accuracy: 1.000000, train loss: 0.006483
Epoch :20 | step 1900, training accuracy: 0.980000, train loss: 0.042864
Epoch :20 | step 2000, training accuracy: 0.990000, train loss: 0.007187
Time : 1 : 6: :47.558219
Test Acc : 0.961711, Test Loss :0.246937
Epoch :21 | step 0, training accuracy: 1.000000, train loss: 0.005663
Epoch :21 | step 100, training accuracy: 1.000000, train loss: 0.013497
Epoch :21 | step 200, training accuracy: 0.990000, train loss: 0.019632
Epoch :21 | step 300, training accuracy: 1.000000, train loss: 0.014248
Epoch :21 | step 400, training accuracy: 0.990000, train loss: 0.015905
Epoch :21 | step 500, training accuracy: 1.000000, train loss: 0.005820
Epoch :21 | step 600, training accuracy: 1.000000, train loss: 0.003034
Epoch :21 | step 700, training accuracy: 1.000000, train loss: 0.002098
Epoch :21 | step 800, training accuracy: 1.000000, train loss: 0.012370
Epoch :21 | step 900, training accuracy: 0.990000, train loss: 0.0686

Epoch :26 | step 100, training accuracy: 0.990000, train loss: 0.029681
Epoch :26 | step 200, training accuracy: 0.990000, train loss: 0.012939
Epoch :26 | step 300, training accuracy: 1.000000, train loss: 0.001722
Epoch :26 | step 400, training accuracy: 1.000000, train loss: 0.002149
Epoch :26 | step 500, training accuracy: 0.990000, train loss: 0.035729
Epoch :26 | step 600, training accuracy: 1.000000, train loss: 0.001403
Epoch :26 | step 700, training accuracy: 1.000000, train loss: 0.004380
Epoch :26 | step 800, training accuracy: 1.000000, train loss: 0.009449
Epoch :26 | step 900, training accuracy: 0.990000, train loss: 0.017788
Epoch :26 | step 1000, training accuracy: 0.980000, train loss: 0.033466
Epoch :26 | step 1100, training accuracy: 1.000000, train loss: 0.001633
Epoch :26 | step 1200, training accuracy: 1.000000, train loss: 0.009373
Epoch :26 | step 1300, training accuracy: 0.990000, train loss: 0.013920
Epoch :26 | step 1400, training accuracy: 0.990000, train lo

Epoch :31 | step 500, training accuracy: 1.000000, train loss: 0.006486
Epoch :31 | step 600, training accuracy: 0.980000, train loss: 0.050329
Epoch :31 | step 700, training accuracy: 1.000000, train loss: 0.008636
Epoch :31 | step 800, training accuracy: 1.000000, train loss: 0.010241
Epoch :31 | step 900, training accuracy: 1.000000, train loss: 0.001156
Epoch :31 | step 1000, training accuracy: 0.990000, train loss: 0.083986
Epoch :31 | step 1100, training accuracy: 0.990000, train loss: 0.068809
Epoch :31 | step 1200, training accuracy: 1.000000, train loss: 0.002790
Epoch :31 | step 1300, training accuracy: 1.000000, train loss: 0.002890
Epoch :31 | step 1400, training accuracy: 1.000000, train loss: 0.000519
Epoch :31 | step 1500, training accuracy: 1.000000, train loss: 0.002661
Epoch :31 | step 1600, training accuracy: 1.000000, train loss: 0.003977
Epoch :31 | step 1700, training accuracy: 1.000000, train loss: 0.000164
Epoch :31 | step 1800, training accuracy: 1.000000, trai

Epoch :36 | step 900, training accuracy: 1.000000, train loss: 0.007431
Epoch :36 | step 1000, training accuracy: 1.000000, train loss: 0.000076
Epoch :36 | step 1100, training accuracy: 1.000000, train loss: 0.000044
Epoch :36 | step 1200, training accuracy: 0.990000, train loss: 0.008798
Epoch :36 | step 1300, training accuracy: 1.000000, train loss: 0.000110
Epoch :36 | step 1400, training accuracy: 1.000000, train loss: 0.001025
Epoch :36 | step 1500, training accuracy: 1.000000, train loss: 0.000064
Epoch :36 | step 1600, training accuracy: 1.000000, train loss: 0.005623
Epoch :36 | step 1700, training accuracy: 1.000000, train loss: 0.000150
Epoch :36 | step 1800, training accuracy: 1.000000, train loss: 0.001400
Epoch :36 | step 1900, training accuracy: 0.990000, train loss: 0.021456
Epoch :36 | step 2000, training accuracy: 1.000000, train loss: 0.000656
Time : 1 : 56: :45.253677
Test Acc : 0.962822, Test Loss :0.292085
Epoch :37 | step 0, training accuracy: 1.000000, train los

Epoch :41 | step 1300, training accuracy: 0.980000, train loss: 0.129219
Epoch :41 | step 1400, training accuracy: 1.000000, train loss: 0.002742
Epoch :41 | step 1500, training accuracy: 1.000000, train loss: 0.000422
Epoch :41 | step 1600, training accuracy: 1.000000, train loss: 0.000164
Epoch :41 | step 1700, training accuracy: 1.000000, train loss: 0.001454
Epoch :41 | step 1800, training accuracy: 0.990000, train loss: 0.013560
Epoch :41 | step 1900, training accuracy: 0.990000, train loss: 0.068127
Epoch :41 | step 2000, training accuracy: 1.000000, train loss: 0.001872
Time : 2 : 12: :23.026315
Test Acc : 0.960002, Test Loss :0.335720
Epoch :42 | step 0, training accuracy: 1.000000, train loss: 0.008225
Epoch :42 | step 100, training accuracy: 1.000000, train loss: 0.000255
Epoch :42 | step 200, training accuracy: 1.000000, train loss: 0.000171
Epoch :42 | step 300, training accuracy: 1.000000, train loss: 0.000371
Epoch :42 | step 400, training accuracy: 1.000000, train loss: 

Epoch :46 | step 1700, training accuracy: 1.000000, train loss: 0.002469
Epoch :46 | step 1800, training accuracy: 1.000000, train loss: 0.000038
Epoch :46 | step 1900, training accuracy: 0.990000, train loss: 0.012262
Epoch :46 | step 2000, training accuracy: 1.000000, train loss: 0.000081
Time : 2 : 28: :0.308924
Test Acc : 0.961925, Test Loss :0.341857
Epoch :47 | step 0, training accuracy: 1.000000, train loss: 0.000337
Epoch :47 | step 100, training accuracy: 1.000000, train loss: 0.000729
Epoch :47 | step 200, training accuracy: 1.000000, train loss: 0.000695
Epoch :47 | step 300, training accuracy: 0.990000, train loss: 0.012447
Epoch :47 | step 400, training accuracy: 1.000000, train loss: 0.001085
Epoch :47 | step 500, training accuracy: 1.000000, train loss: 0.000105
Epoch :47 | step 600, training accuracy: 0.990000, train loss: 0.007247
Epoch :47 | step 700, training accuracy: 1.000000, train loss: 0.000243
Epoch :47 | step 800, training accuracy: 1.000000, train loss: 0.000

In [16]:
train_file_name = "Shuffled_with_alphabet_HanDB_Train.tfrecord"
test_file_name = "Shuffled_with_alphabet_HanDB_Test.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)


train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label, train_cho, train_jung, train_jong = train_iterator.get_next()
test_image, test_label, test_cho, test_jung, test_jong = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [17]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    merged_summary2 = tf.summary.merge([loss_summ, acc_summ,cho_summ,jung_summ,jong_summ])
    train_writer = tf.summary.FileWriter("./logs/CNNV2/HanDB/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV2/HanDB/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    
    #Train Model
    
    start_time = time.time()
    global_step=0
    for epoch in range(50):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_cho, batch_jung, batch_jong = sess.run([train_image, train_cho, train_jung, train_jong])
            
            sess.run(optimizer,feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:True})
            train_acc, train_loss, train_cho_acc, train_jung_acc, train_jong_acc = sess.run([accuracy, total_loss,cho_acc, jung_acc, jong_acc ], feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                print("  **Acc cho: %f, Acc jung: %f, Acc jong: %f **"% (train_cho_acc,train_jung_acc, train_jong_acc))
                summary = sess.run(merged_summary2, feed_dict={input_images: batch_images, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
                
        
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        save_path = saver.save(sess, "save/HanDB/CNNV2/CNNV2_HanDB.ckpt")
        
        acc_sum=0.0
        loss_sum=0.0
        cho_sum=0.0
        jung_sum=0.0
        jong_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_cho, test_jung, test_jong])
            try:
                test_acc, test_loss, test_cho_acc, test_jung_acc, test_jong_acc = sess.run([test_accuracy, total_loss, batch_test_cho_acc, batch_test_jung_acc, batch_test_jong_acc ], feed_dict={input_images: batch_images, input_label_cho:batch_cho, 
                                                               input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})

            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
            cho_sum+=test_cho_acc
            jung_sum+=test_jung_acc
            jong_sum+=test_jong_acc
            loss_sum+=test_loss*len(batch_cho)
            
        test_loss=loss_sum/test_data_length
        test_acc = acc_sum/test_data_length
        test_cho_acc = cho_sum/test_data_length
        test_jung_acc = jung_sum/test_data_length
        test_jong_acc = jong_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
        print("  Acc cho: %f, Acc jung: %f, Acc jong: %f "% (test_cho_acc,test_jung_acc, test_jong_acc))
        
    print("Training is done.")


    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(threads)

Train total Batch :6658
Epoch :0 | step 0, training accuracy: 0.000000, train loss: 9.186160
  **Acc cho: 0.040000, Acc jung: 0.020000, Acc jong: 0.050000 **
Epoch :0 | step 100, training accuracy: 0.000000, train loss: 8.411021
  **Acc cho: 0.110000, Acc jung: 0.070000, Acc jong: 0.310000 **
Epoch :0 | step 200, training accuracy: 0.090000, train loss: 5.795107
  **Acc cho: 0.350000, Acc jung: 0.400000, Acc jong: 0.740000 **
Epoch :0 | step 300, training accuracy: 0.290000, train loss: 3.838552
  **Acc cho: 0.600000, Acc jung: 0.610000, Acc jong: 0.810000 **
Epoch :0 | step 400, training accuracy: 0.340000, train loss: 3.015432
  **Acc cho: 0.650000, Acc jung: 0.740000, Acc jong: 0.770000 **
Epoch :0 | step 500, training accuracy: 0.230000, train loss: 3.674941
  **Acc cho: 0.490000, Acc jung: 0.640000, Acc jong: 0.810000 **
Epoch :0 | step 600, training accuracy: 0.450000, train loss: 2.261279
  **Acc cho: 0.750000, Acc jung: 0.790000, Acc jong: 0.770000 **
Epoch :0 | step 700, train

Epoch :0 | step 6000, training accuracy: 0.920000, train loss: 0.358231
  **Acc cho: 0.950000, Acc jung: 0.980000, Acc jong: 0.980000 **
Epoch :0 | step 6100, training accuracy: 0.910000, train loss: 0.552450
  **Acc cho: 0.970000, Acc jung: 0.960000, Acc jong: 0.960000 **
Epoch :0 | step 6200, training accuracy: 0.860000, train loss: 0.374705
  **Acc cho: 0.970000, Acc jung: 0.960000, Acc jong: 0.910000 **
Epoch :0 | step 6300, training accuracy: 0.910000, train loss: 0.401054
  **Acc cho: 0.940000, Acc jung: 0.980000, Acc jong: 0.970000 **
Epoch :0 | step 6400, training accuracy: 0.940000, train loss: 0.185242
  **Acc cho: 0.970000, Acc jung: 0.990000, Acc jong: 0.980000 **
Epoch :0 | step 6500, training accuracy: 0.890000, train loss: 0.397087
  **Acc cho: 0.960000, Acc jung: 0.970000, Acc jong: 0.960000 **
Epoch :0 | step 6600, training accuracy: 0.930000, train loss: 0.276640
  **Acc cho: 0.980000, Acc jung: 0.980000, Acc jong: 0.960000 **
Time : 0 : 9: :57.631183
Test Acc : 0.917

KeyboardInterrupt: 

In [14]:
W2_cho2 = tf.Variable(tf.random_normal([512,19],stddev=0.01))
W2_jung2 = tf.Variable(tf.random_normal([512,21],stddev=0.01))
W2_jong2 = tf.Variable(tf.random_normal([512,28],stddev=0.01))

print(fc)
fc2_cho2 = tf.matmul(fc, W2_cho2)
cho_logit = fc2_cho2
bn2_cho2 = tf.layers.batch_normalization(fc2_cho2, training=is_training)
fc2_cho2 = tf.nn.relu(bn2_cho2)
fc2_cho2 = tf.nn.dropout(fc2_cho2, keep_prob)

fc2_jung2 = tf.matmul(fc, W2_jung2)
jung_logit = fc2_jung2
bn2_jung2 = tf.layers.batch_normalization(fc2_jung2, training=is_training)
fc2_jung2 = tf.nn.relu(bn2_jung2)
fc2_jung2 = tf.nn.dropout(fc2_jung2, keep_prob)

fc2_jong2 = tf.matmul(fc, W2_jong2)
jong_logit = fc2_jong2
bn2_jong2 = tf.layers.batch_normalization(fc2_jong2, training=is_training)
fc2_jong2 = tf.nn.relu(bn2_jong2)
fc2_jong2 = tf.nn.dropout(fc2_jong2, keep_prob)

Tensor("Reshape:0", shape=(?, 512), dtype=float32)


In [15]:
W_cho3 = tf.Variable(tf.random_normal([19,2350],stddev=0.01))
W_jung3 = tf.Variable(tf.random_normal([21,2350],stddev=0.01))
W_jong3 = tf.Variable(tf.random_normal([28,2350],stddev=0.01))

fc3_cho2 = tf.matmul(fc2_cho2, W_cho3)
fc3_jung2 = tf.matmul(fc2_jung2, W_jung3)
fc3_jong2 = tf.matmul(fc2_jong2, W_jong3)

logit = fc3_cho2 + fc3_jung2 + fc3_jong2

In [16]:
cross_entropy2 = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=input_labels)
loss = tf.reduce_mean(cross_entropy2)

cho_cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=cho_logit, labels=input_label_cho)
jung_cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=jung_logit, labels=input_label_jung)
jong_cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=jong_logit, labels=input_label_jong)

cho_loss = tf.reduce_mean(cho_cross_entropy)
jung_loss = tf.reduce_mean(jung_cross_entropy)
jong_loss = tf.reduce_mean(jong_cross_entropy)

loss = loss + 3*((0.4*cho_loss) + 0.4*(jung_loss) + 0.2*(jong_loss))

update_ops2 = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops2):
    optimizer2 = tf.train.AdamOptimizer(0.001).minimize(loss)
loss_summ2=tf.summary.scalar("Loss",loss)
        
correct_prediction2 = tf.equal(tf.argmax(logit,1), input_labels)
accuracy2 = tf.reduce_mean(tf.cast(correct_prediction2, tf.float32))
test_accuracy2 = tf.reduce_sum(tf.cast(correct_prediction2, tf.float32))
acc_summ2=tf.summary.scalar("Accuracy",accuracy2)

In [17]:
def decode2(serialized_example):
    features = tf.parse_single_example(
      serialized_example,      
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'width': tf.FixedLenFeature([], tf.int64),
          'height': tf.FixedLenFeature([], tf.int64),
          'image': tf.FixedLenFeature([], tf.string)
      })
    image = tf.decode_raw(features['image'],tf.uint8)
    #image = tf.reshape(image,[64,64,1])
    image = tf.cast(image, tf.float32)
    image = tf.reshape(image, [64,64,1])
    label = tf.cast(features['label'],tf.int64)
    
    return image,label

In [18]:
batch_size=100
#test_file_name = "Shuffled_PE92_Test.tfrecord"
#train_file_name = "Shuffled_PE92_Train.tfrecord"

test_file_name = "Shuffled_with_alphabet_PE92_Test.tfrecord"
train_file_name = "Shuffled_with_alphabet_PE92_Train.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label, train_cho, train_jung, train_jong = train_iterator.get_next()
test_image, test_label, test_cho, test_jung, test_jong = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [19]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    merged_summary3 = tf.summary.merge([loss_summ2, acc_summ2])
    train_writer = tf.summary.FileWriter("./logs/CNNV3/PE92/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV3/PE92/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)
    #Train Model
    
    start_time = time.time()
    global_step=0
    for epoch in range(50):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels, batch_cho, batch_jung, batch_jong = sess.run([train_image, train_label, train_cho, train_jung, train_jong])
            
            sess.run(optimizer2,feed_dict={input_images: batch_images, input_labels: batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:True})
            train_acc, train_loss=sess.run([accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary3, feed_dict={input_images: batch_images, input_labels:batch_labels, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
        
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        save_path = saver.save(sess, "save/PE92/CNNV3/CNNV3_PE92.ckpt")
        acc_sum=0.0
        loss_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_labels, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_label, test_cho, test_jung, test_jong])
            try:
                test_acc, test_loss=sess.run([test_accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong,keep_prob:1.0, is_training:False})
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
            loss_sum+=test_loss*len(batch_labels)
        loss_sum=loss_sum/test_data_length
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
       
    print("Training is done.")

    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(threads)

Train total Batch :2002
Epoch :0 | step 0, training accuracy: 0.000000, train loss: 16.868673
Epoch :0 | step 100, training accuracy: 0.000000, train loss: 23.276392
Epoch :0 | step 200, training accuracy: 0.000000, train loss: 22.433615
Epoch :0 | step 300, training accuracy: 0.000000, train loss: 14.543561
Epoch :0 | step 400, training accuracy: 0.030000, train loss: 8.271237
Epoch :0 | step 500, training accuracy: 0.230000, train loss: 5.318247
Epoch :0 | step 600, training accuracy: 0.370000, train loss: 3.773086
Epoch :0 | step 700, training accuracy: 0.130000, train loss: 7.605534
Epoch :0 | step 800, training accuracy: 0.570000, train loss: 3.013556
Epoch :0 | step 900, training accuracy: 0.610000, train loss: 2.417877
Epoch :0 | step 1000, training accuracy: 0.400000, train loss: 3.473219
Epoch :0 | step 1100, training accuracy: 0.300000, train loss: 5.108683
Epoch :0 | step 1200, training accuracy: 0.780000, train loss: 1.706753
Epoch :0 | step 1300, training accuracy: 0.35000

Epoch :5 | step 500, training accuracy: 0.990000, train loss: 0.163401
Epoch :5 | step 600, training accuracy: 1.000000, train loss: 0.171556
Epoch :5 | step 700, training accuracy: 0.980000, train loss: 0.183028
Epoch :5 | step 800, training accuracy: 0.970000, train loss: 0.186880
Epoch :5 | step 900, training accuracy: 0.950000, train loss: 0.281871
Epoch :5 | step 1000, training accuracy: 0.980000, train loss: 0.225988
Epoch :5 | step 1100, training accuracy: 0.980000, train loss: 0.167967
Epoch :5 | step 1200, training accuracy: 0.990000, train loss: 0.074961
Epoch :5 | step 1300, training accuracy: 0.980000, train loss: 0.107157
Epoch :5 | step 1400, training accuracy: 0.990000, train loss: 0.139790
Epoch :5 | step 1500, training accuracy: 1.000000, train loss: 0.100119
Epoch :5 | step 1600, training accuracy: 0.970000, train loss: 0.210169
Epoch :5 | step 1700, training accuracy: 0.990000, train loss: 0.171469
Epoch :5 | step 1800, training accuracy: 0.990000, train loss: 0.0998

Epoch :10 | step 1000, training accuracy: 0.990000, train loss: 0.138798
Epoch :10 | step 1100, training accuracy: 1.000000, train loss: 0.050670
Epoch :10 | step 1200, training accuracy: 0.980000, train loss: 0.035073
Epoch :10 | step 1300, training accuracy: 0.990000, train loss: 0.074552
Epoch :10 | step 1400, training accuracy: 0.990000, train loss: 0.064221
Epoch :10 | step 1500, training accuracy: 0.990000, train loss: 0.078819
Epoch :10 | step 1600, training accuracy: 1.000000, train loss: 0.030502
Epoch :10 | step 1700, training accuracy: 1.000000, train loss: 0.043889
Epoch :10 | step 1800, training accuracy: 0.990000, train loss: 0.059994
Epoch :10 | step 1900, training accuracy: 0.990000, train loss: 0.037421
Epoch :10 | step 2000, training accuracy: 0.990000, train loss: 0.017108
Time : 0 : 35: :35.523145
Test Acc : 0.972651, Test Loss :0.818948
Epoch :11 | step 0, training accuracy: 0.980000, train loss: 0.110358
Epoch :11 | step 100, training accuracy: 1.000000, train los

Epoch :15 | step 1400, training accuracy: 1.000000, train loss: 0.022473
Epoch :15 | step 1500, training accuracy: 1.000000, train loss: 0.041522
Epoch :15 | step 1600, training accuracy: 1.000000, train loss: 0.008224
Epoch :15 | step 1700, training accuracy: 0.980000, train loss: 0.034455
Epoch :15 | step 1800, training accuracy: 1.000000, train loss: 0.021243
Epoch :15 | step 1900, training accuracy: 1.000000, train loss: 0.013999
Epoch :15 | step 2000, training accuracy: 0.990000, train loss: 0.120591
Time : 0 : 51: :41.987423
Test Acc : 0.964617, Test Loss :0.005172
Epoch :16 | step 0, training accuracy: 0.980000, train loss: 0.066597
Epoch :16 | step 100, training accuracy: 1.000000, train loss: 0.024096
Epoch :16 | step 200, training accuracy: 1.000000, train loss: 0.047878
Epoch :16 | step 300, training accuracy: 0.990000, train loss: 0.022427
Epoch :16 | step 400, training accuracy: 1.000000, train loss: 0.020128
Epoch :16 | step 500, training accuracy: 1.000000, train loss: 0

Epoch :20 | step 1800, training accuracy: 1.000000, train loss: 0.002823
Epoch :20 | step 1900, training accuracy: 1.000000, train loss: 0.012115
Epoch :20 | step 2000, training accuracy: 1.000000, train loss: 0.028463
Time : 1 : 7: :45.827552
Test Acc : 0.973463, Test Loss :0.161883
Epoch :21 | step 0, training accuracy: 1.000000, train loss: 0.005559
Epoch :21 | step 100, training accuracy: 1.000000, train loss: 0.005013
Epoch :21 | step 200, training accuracy: 1.000000, train loss: 0.007056
Epoch :21 | step 300, training accuracy: 1.000000, train loss: 0.004964
Epoch :21 | step 400, training accuracy: 1.000000, train loss: 0.005942
Epoch :21 | step 500, training accuracy: 1.000000, train loss: 0.008897
Epoch :21 | step 600, training accuracy: 1.000000, train loss: 0.003267
Epoch :21 | step 700, training accuracy: 1.000000, train loss: 0.004939
Epoch :21 | step 800, training accuracy: 1.000000, train loss: 0.042250
Epoch :21 | step 900, training accuracy: 1.000000, train loss: 0.0058

Epoch :26 | step 100, training accuracy: 1.000000, train loss: 0.006791
Epoch :26 | step 200, training accuracy: 1.000000, train loss: 0.003970
Epoch :26 | step 300, training accuracy: 0.990000, train loss: 0.018037
Epoch :26 | step 400, training accuracy: 1.000000, train loss: 0.022372
Epoch :26 | step 500, training accuracy: 0.990000, train loss: 0.095269
Epoch :26 | step 600, training accuracy: 1.000000, train loss: 0.020165
Epoch :26 | step 700, training accuracy: 1.000000, train loss: 0.010203
Epoch :26 | step 800, training accuracy: 1.000000, train loss: 0.006650
Epoch :26 | step 900, training accuracy: 1.000000, train loss: 0.004552
Epoch :26 | step 1000, training accuracy: 1.000000, train loss: 0.004757
Epoch :26 | step 1100, training accuracy: 0.990000, train loss: 0.017746
Epoch :26 | step 1200, training accuracy: 1.000000, train loss: 0.034557
Epoch :26 | step 1300, training accuracy: 1.000000, train loss: 0.003182
Epoch :26 | step 1400, training accuracy: 1.000000, train lo

Epoch :31 | step 500, training accuracy: 1.000000, train loss: 0.002385
Epoch :31 | step 600, training accuracy: 1.000000, train loss: 0.000723
Epoch :31 | step 700, training accuracy: 1.000000, train loss: 0.005422
Epoch :31 | step 800, training accuracy: 1.000000, train loss: 0.016889
Epoch :31 | step 900, training accuracy: 1.000000, train loss: 0.012257
Epoch :31 | step 1000, training accuracy: 1.000000, train loss: 0.003125
Epoch :31 | step 1100, training accuracy: 1.000000, train loss: 0.004591
Epoch :31 | step 1200, training accuracy: 1.000000, train loss: 0.001331
Epoch :31 | step 1300, training accuracy: 1.000000, train loss: 0.006635
Epoch :31 | step 1400, training accuracy: 1.000000, train loss: 0.006585
Epoch :31 | step 1500, training accuracy: 0.990000, train loss: 0.019332
Epoch :31 | step 1600, training accuracy: 1.000000, train loss: 0.004249
Epoch :31 | step 1700, training accuracy: 0.990000, train loss: 0.188980
Epoch :31 | step 1800, training accuracy: 1.000000, trai

Epoch :36 | step 900, training accuracy: 1.000000, train loss: 0.001383
Epoch :36 | step 1000, training accuracy: 0.990000, train loss: 0.244378
Epoch :36 | step 1100, training accuracy: 1.000000, train loss: 0.004471
Epoch :36 | step 1200, training accuracy: 1.000000, train loss: 0.011420
Epoch :36 | step 1300, training accuracy: 1.000000, train loss: 0.003088
Epoch :36 | step 1400, training accuracy: 1.000000, train loss: 0.003598
Epoch :36 | step 1500, training accuracy: 1.000000, train loss: 0.004384
Epoch :36 | step 1600, training accuracy: 1.000000, train loss: 0.001848
Epoch :36 | step 1700, training accuracy: 1.000000, train loss: 0.003517
Epoch :36 | step 1800, training accuracy: 1.000000, train loss: 0.000906
Epoch :36 | step 1900, training accuracy: 0.990000, train loss: 0.018120
Epoch :36 | step 2000, training accuracy: 0.990000, train loss: 0.032563
Time : 1 : 59: :10.008957
Test Acc : 0.974745, Test Loss :0.215550
Epoch :37 | step 0, training accuracy: 1.000000, train los

Epoch :41 | step 1300, training accuracy: 1.000000, train loss: 0.000793
Epoch :41 | step 1400, training accuracy: 1.000000, train loss: 0.001156
Epoch :41 | step 1500, training accuracy: 1.000000, train loss: 0.001727
Epoch :41 | step 1600, training accuracy: 1.000000, train loss: 0.000335
Epoch :41 | step 1700, training accuracy: 1.000000, train loss: 0.002077
Epoch :41 | step 1800, training accuracy: 1.000000, train loss: 0.001658
Epoch :41 | step 1900, training accuracy: 1.000000, train loss: 0.011763
Epoch :41 | step 2000, training accuracy: 1.000000, train loss: 0.000764
Time : 2 : 15: :14.282110
Test Acc : 0.974873, Test Loss :0.000025
Epoch :42 | step 0, training accuracy: 1.000000, train loss: 0.002015
Epoch :42 | step 100, training accuracy: 1.000000, train loss: 0.002385
Epoch :42 | step 200, training accuracy: 1.000000, train loss: 0.008505
Epoch :42 | step 300, training accuracy: 1.000000, train loss: 0.006694
Epoch :42 | step 400, training accuracy: 1.000000, train loss: 

Epoch :46 | step 1700, training accuracy: 1.000000, train loss: 0.001858
Epoch :46 | step 1800, training accuracy: 1.000000, train loss: 0.000686
Epoch :46 | step 1900, training accuracy: 1.000000, train loss: 0.000214
Epoch :46 | step 2000, training accuracy: 1.000000, train loss: 0.009692
Time : 2 : 31: :18.252246
Test Acc : 0.975984, Test Loss :1.412243
Epoch :47 | step 0, training accuracy: 1.000000, train loss: 0.039423
Epoch :47 | step 100, training accuracy: 1.000000, train loss: 0.000652
Epoch :47 | step 200, training accuracy: 1.000000, train loss: 0.000620
Epoch :47 | step 300, training accuracy: 1.000000, train loss: 0.000163
Epoch :47 | step 400, training accuracy: 1.000000, train loss: 0.023388
Epoch :47 | step 500, training accuracy: 1.000000, train loss: 0.018383
Epoch :47 | step 600, training accuracy: 1.000000, train loss: 0.004638
Epoch :47 | step 700, training accuracy: 1.000000, train loss: 0.001909
Epoch :47 | step 800, training accuracy: 1.000000, train loss: 0.00

In [20]:
train_file_name = "Shuffled_with_alphabet_HanDB_Train.tfrecord"
test_file_name = "Shuffled_with_alphabet_HanDB_Test.tfrecord"

train_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+train_file_name))
test_data_length = sum(1 for _ in tf.python_io.tf_record_iterator("./"+test_file_name))
total_test_batch = int(test_data_length/batch_size)+1

train_dataset = tf.data.TFRecordDataset("./"+train_file_name)
train_dataset = train_dataset.map(decode)
train_dataset = train_dataset.repeat()
train_dataset= train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size)

test_dataset = tf.data.TFRecordDataset("./"+test_file_name)
test_dataset = test_dataset.map(decode)
#test_dataset = test_dataset.repeat()
#test_dataset= test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size)


train_iterator = tf.data.Iterator.from_structure(train_dataset.output_types,
                                           train_dataset.output_shapes)
test_iterator = tf.data.Iterator.from_structure(test_dataset.output_types,
                                           test_dataset.output_shapes)

train_init_op = train_iterator.make_initializer(train_dataset)
test_init_op = test_iterator.make_initializer(test_dataset)

train_image, train_label, train_cho, train_jung, train_jong = train_iterator.get_next()
test_image, test_label, test_cho, test_jung, test_jong = test_iterator.get_next()

train_total_batch = int(train_data_length/batch_size)
test_total_batch = int(test_data_length/batch_size)

In [21]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())
saver = tf.train.Saver()

print("Train total Batch :%d" % (train_total_batch))
with tf.Session() as sess:
    merged_summary4 = tf.summary.merge([loss_summ2, acc_summ2])
    train_writer = tf.summary.FileWriter("./logs/CNNV3/HanDB/Train")
    test_writer = tf.summary.FileWriter("./logs/CNNV3/HanDB/Test")
    
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    sess.run(train_init_op)
    sess.run(test_init_op)
    #Train Model
    
    start_time = time.time()
    global_step=0
    for epoch in range(50):
        for i in range(train_total_batch):
        #for i in range(batch_size):
            batch_images, batch_labels, batch_cho, batch_jung, batch_jong = sess.run([train_image, train_label, train_cho, train_jung, train_jong])
            
            sess.run(optimizer2,feed_dict={input_images: batch_images, input_labels: batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:0.5, is_training:True})
            train_acc, train_loss=sess.run([accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
            
            if(i%100==0):
                print('Epoch :%d | step %d, training accuracy: %f, train loss: %f' % (epoch,i, train_acc,train_loss))
                summary = sess.run(merged_summary4, feed_dict={input_images: batch_images, input_labels:batch_labels, input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong, keep_prob:1.0, is_training:False})
                train_writer.add_summary(summary,global_step)
                global_step+=1
        
        end_time = time.time()
        elapsed = end_time - start_time
        print("Time : %d : %d: :%f" %(int(elapsed/3600), int((elapsed%3600)/60), (elapsed%60)))
        save_path = saver.save(sess, "save/HanDB/CNNV3/CNNV3_PE92.ckpt")
        
        acc_sum=0.0
        loss_sum=0.0
        sess.run(test_init_op)
        for epoch in range(total_test_batch):
            batch_images, batch_labels, batch_cho, batch_jung, batch_jong = sess.run([test_image, test_label, test_cho, test_jung, test_jong])
            try:
                test_acc, test_loss=sess.run([test_accuracy2, loss], feed_dict={input_images: batch_images, input_labels:batch_labels,input_label_cho:batch_cho, input_label_jung : batch_jung, input_label_jong : batch_jong,keep_prob:1.0, is_training:False})
            except tf.errors.OutOfRangeError:
                print("Out of Data ierators")
            acc_sum+=test_acc
            loss_sum+=test_loss*len(batch_labels)
        loss_sum=loss_sum/test_data_length
        test_acc = acc_sum/test_data_length
        print("Test Acc : %f, Test Loss :%f" %(test_acc,test_loss))
       
    print("Training is done.")

    train_writer.close()
    test_writer.close()
    coord.request_stop()
    coord.join(threads)

Train total Batch :6658
Epoch :0 | step 0, training accuracy: 0.000000, train loss: 16.836971
Epoch :0 | step 100, training accuracy: 0.010000, train loss: 15.993170
Epoch :0 | step 200, training accuracy: 0.000000, train loss: 17.554955
Epoch :0 | step 300, training accuracy: 0.030000, train loss: 12.320048
Epoch :0 | step 400, training accuracy: 0.470000, train loss: 6.323279
Epoch :0 | step 500, training accuracy: 0.450000, train loss: 5.909383
Epoch :0 | step 600, training accuracy: 0.130000, train loss: 10.172680
Epoch :0 | step 700, training accuracy: 0.560000, train loss: 5.324970
Epoch :0 | step 800, training accuracy: 0.520000, train loss: 6.166737
Epoch :0 | step 900, training accuracy: 0.650000, train loss: 3.114682
Epoch :0 | step 1000, training accuracy: 0.750000, train loss: 2.688135
Epoch :0 | step 1100, training accuracy: 0.710000, train loss: 2.664098
Epoch :0 | step 1200, training accuracy: 0.690000, train loss: 2.501263
Epoch :0 | step 1300, training accuracy: 0.6500

Epoch :1 | step 4600, training accuracy: 0.920000, train loss: 0.871052
Epoch :1 | step 4700, training accuracy: 0.960000, train loss: 0.245179
Epoch :1 | step 4800, training accuracy: 0.970000, train loss: 0.310011
Epoch :1 | step 4900, training accuracy: 0.970000, train loss: 0.414500
Epoch :1 | step 5000, training accuracy: 0.960000, train loss: 0.198959
Epoch :1 | step 5100, training accuracy: 0.980000, train loss: 0.323615
Epoch :1 | step 5200, training accuracy: 0.960000, train loss: 0.487208
Epoch :1 | step 5300, training accuracy: 0.930000, train loss: 0.527260
Epoch :1 | step 5400, training accuracy: 0.920000, train loss: 1.020287
Epoch :1 | step 5500, training accuracy: 0.960000, train loss: 0.321047
Epoch :1 | step 5600, training accuracy: 0.980000, train loss: 0.220148
Epoch :1 | step 5700, training accuracy: 0.970000, train loss: 0.353796
Epoch :1 | step 5800, training accuracy: 0.940000, train loss: 0.422360
Epoch :1 | step 5900, training accuracy: 0.900000, train loss: 0

Epoch :3 | step 2500, training accuracy: 0.990000, train loss: 0.092545
Epoch :3 | step 2600, training accuracy: 0.980000, train loss: 0.148237
Epoch :3 | step 2700, training accuracy: 0.970000, train loss: 0.234017
Epoch :3 | step 2800, training accuracy: 0.990000, train loss: 0.265068
Epoch :3 | step 2900, training accuracy: 0.960000, train loss: 0.201671
Epoch :3 | step 3000, training accuracy: 0.980000, train loss: 0.196363
Epoch :3 | step 3100, training accuracy: 0.980000, train loss: 0.294456
Epoch :3 | step 3200, training accuracy: 1.000000, train loss: 0.049361
Epoch :3 | step 3300, training accuracy: 0.970000, train loss: 0.281305
Epoch :3 | step 3400, training accuracy: 0.980000, train loss: 0.074845
Epoch :3 | step 3500, training accuracy: 0.970000, train loss: 0.300341
Epoch :3 | step 3600, training accuracy: 0.990000, train loss: 0.146055
Epoch :3 | step 3700, training accuracy: 1.000000, train loss: 0.110041
Epoch :3 | step 3800, training accuracy: 0.990000, train loss: 0

Epoch :5 | step 400, training accuracy: 0.990000, train loss: 0.110929
Epoch :5 | step 500, training accuracy: 0.980000, train loss: 0.082206
Epoch :5 | step 600, training accuracy: 0.990000, train loss: 0.068244
Epoch :5 | step 700, training accuracy: 0.970000, train loss: 0.122117
Epoch :5 | step 800, training accuracy: 0.970000, train loss: 0.174387
Epoch :5 | step 900, training accuracy: 0.990000, train loss: 0.079780
Epoch :5 | step 1000, training accuracy: 0.970000, train loss: 0.177599
Epoch :5 | step 1100, training accuracy: 0.990000, train loss: 0.172886
Epoch :5 | step 1200, training accuracy: 0.990000, train loss: 0.093051
Epoch :5 | step 1300, training accuracy: 0.980000, train loss: 0.080232
Epoch :5 | step 1400, training accuracy: 0.990000, train loss: 0.133636
Epoch :5 | step 1500, training accuracy: 0.980000, train loss: 0.136207
Epoch :5 | step 1600, training accuracy: 0.990000, train loss: 0.161018
Epoch :5 | step 1700, training accuracy: 1.000000, train loss: 0.02996

Epoch :6 | step 5100, training accuracy: 1.000000, train loss: 0.073140
Epoch :6 | step 5200, training accuracy: 0.990000, train loss: 0.204955
Epoch :6 | step 5300, training accuracy: 0.960000, train loss: 0.308735
Epoch :6 | step 5400, training accuracy: 0.990000, train loss: 0.291303
Epoch :6 | step 5500, training accuracy: 0.980000, train loss: 0.201046
Epoch :6 | step 5600, training accuracy: 0.980000, train loss: 0.106775
Epoch :6 | step 5700, training accuracy: 1.000000, train loss: 0.026896
Epoch :6 | step 5800, training accuracy: 1.000000, train loss: 0.112886
Epoch :6 | step 5900, training accuracy: 1.000000, train loss: 0.031259
Epoch :6 | step 6000, training accuracy: 0.990000, train loss: 0.058617
Epoch :6 | step 6100, training accuracy: 1.000000, train loss: 0.014966
Epoch :6 | step 6200, training accuracy: 0.970000, train loss: 0.282527
Epoch :6 | step 6300, training accuracy: 1.000000, train loss: 0.062376
Epoch :6 | step 6400, training accuracy: 1.000000, train loss: 0

...

Epoch :41 | step 4800, training accuracy: 1.000000, train loss: 0.003174
Epoch :41 | step 4900, training accuracy: 1.000000, train loss: 0.000950
Epoch :41 | step 5000, training accuracy: 1.000000, train loss: 0.001196
Epoch :41 | step 5100, training accuracy: 1.000000, train loss: 0.001843
Epoch :41 | step 5200, training accuracy: 1.000000, train loss: 0.002198
Epoch :41 | step 5300, training accuracy: 1.000000, train loss: 0.001483
Epoch :41 | step 5400, training accuracy: 1.000000, train loss: 0.002132
Epoch :41 | step 5500, training accuracy: 1.000000, train loss: 0.000240
Epoch :41 | step 5600, training accuracy: 1.000000, train loss: 0.002844
Epoch :41 | step 5700, training accuracy: 1.000000, train loss: 0.010906
Epoch :41 | step 5800, training accuracy: 1.000000, train loss: 0.008926
Epoch :41 | step 5900, training accuracy: 1.000000, train loss: 0.002959
Epoch :41 | step 6000, training accuracy: 1.000000, train loss: 0.015902
Epoch :41 | step 6100, training accuracy: 1.000000,

Epoch :43 | step 2500, training accuracy: 1.000000, train loss: 0.008396
Epoch :43 | step 2600, training accuracy: 1.000000, train loss: 0.008417
Epoch :43 | step 2700, training accuracy: 1.000000, train loss: 0.004768
Epoch :43 | step 2800, training accuracy: 1.000000, train loss: 0.000518
Epoch :43 | step 2900, training accuracy: 1.000000, train loss: 0.000825
Epoch :43 | step 3000, training accuracy: 1.000000, train loss: 0.001082
Epoch :43 | step 3100, training accuracy: 1.000000, train loss: 0.000559
Epoch :43 | step 3200, training accuracy: 1.000000, train loss: 0.005337
Epoch :43 | step 3300, training accuracy: 1.000000, train loss: 0.000915
Epoch :43 | step 3400, training accuracy: 1.000000, train loss: 0.004230
Epoch :43 | step 3500, training accuracy: 1.000000, train loss: 0.007968
Epoch :43 | step 3600, training accuracy: 1.000000, train loss: 0.000931
Epoch :43 | step 3700, training accuracy: 1.000000, train loss: 0.002079
Epoch :43 | step 3800, training accuracy: 1.000000,

Epoch :45 | step 200, training accuracy: 1.000000, train loss: 0.000994
Epoch :45 | step 300, training accuracy: 1.000000, train loss: 0.007525
Epoch :45 | step 400, training accuracy: 1.000000, train loss: 0.007073
Epoch :45 | step 500, training accuracy: 1.000000, train loss: 0.000579
Epoch :45 | step 600, training accuracy: 1.000000, train loss: 0.011948
Epoch :45 | step 700, training accuracy: 1.000000, train loss: 0.010578
Epoch :45 | step 800, training accuracy: 1.000000, train loss: 0.002075
Epoch :45 | step 900, training accuracy: 1.000000, train loss: 0.014258
Epoch :45 | step 1000, training accuracy: 1.000000, train loss: 0.014752
Epoch :45 | step 1100, training accuracy: 1.000000, train loss: 0.003066
Epoch :45 | step 1200, training accuracy: 0.990000, train loss: 0.009347
Epoch :45 | step 1300, training accuracy: 1.000000, train loss: 0.000662
Epoch :45 | step 1400, training accuracy: 1.000000, train loss: 0.002438
Epoch :45 | step 1500, training accuracy: 1.000000, train l

Epoch :46 | step 4700, training accuracy: 1.000000, train loss: 0.002668
Epoch :46 | step 4800, training accuracy: 1.000000, train loss: 0.002654
Epoch :46 | step 4900, training accuracy: 0.990000, train loss: 0.011264
Epoch :46 | step 5000, training accuracy: 1.000000, train loss: 0.017439
Epoch :46 | step 5100, training accuracy: 1.000000, train loss: 0.005892
Epoch :46 | step 5200, training accuracy: 1.000000, train loss: 0.007854
Epoch :46 | step 5300, training accuracy: 1.000000, train loss: 0.002452
Epoch :46 | step 5400, training accuracy: 1.000000, train loss: 0.002611
Epoch :46 | step 5500, training accuracy: 1.000000, train loss: 0.001606
Epoch :46 | step 5600, training accuracy: 1.000000, train loss: 0.001006
Epoch :46 | step 5700, training accuracy: 1.000000, train loss: 0.000147
Epoch :46 | step 5800, training accuracy: 1.000000, train loss: 0.000662
Epoch :46 | step 5900, training accuracy: 1.000000, train loss: 0.001688
Epoch :46 | step 6000, training accuracy: 1.000000,

Epoch :48 | step 2400, training accuracy: 1.000000, train loss: 0.001707
Epoch :48 | step 2500, training accuracy: 1.000000, train loss: 0.001225
Epoch :48 | step 2600, training accuracy: 1.000000, train loss: 0.010548
Epoch :48 | step 2700, training accuracy: 1.000000, train loss: 0.000555
Epoch :48 | step 2800, training accuracy: 1.000000, train loss: 0.000073
Epoch :48 | step 2900, training accuracy: 1.000000, train loss: 0.004016
Epoch :48 | step 3000, training accuracy: 1.000000, train loss: 0.000466
Epoch :48 | step 3100, training accuracy: 0.990000, train loss: 0.027862
Epoch :48 | step 3200, training accuracy: 1.000000, train loss: 0.003110
Epoch :48 | step 3300, training accuracy: 1.000000, train loss: 0.002736
Epoch :48 | step 3400, training accuracy: 1.000000, train loss: 0.001224
Epoch :48 | step 3500, training accuracy: 0.990000, train loss: 0.050896
Epoch :48 | step 3600, training accuracy: 1.000000, train loss: 0.000364
Epoch :48 | step 3700, training accuracy: 1.000000,